In [16]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import numpy as np
from itertools import islice

In [13]:
from gluonts.evaluation import make_evaluation_predictions, Evaluator, MultivariateEvaluator
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

In [3]:
from pts.model.dnri import DNRIEstimator
from pts.modules import StudentTOutput

/home/kashif/.env/pytorch/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [4]:
dataset = get_dataset("exchange_rate_nips", regenerate=False)

In [5]:
dataset.metadata

MetaData(freq='B', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=30)

In [6]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                   max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [7]:
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

/home/kashif/gluon-ts-PR/src/gluonts/dataset/common.py:326: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/home/kashif/gluon-ts-PR/src/gluonts/dataset/common.py:329: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if isinstance(timestamp.freq, Tick):
/home/kashif/gluon-ts-PR/src/gluonts/dataset/common.py:340: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  return timestamp.freq.rollforward(timestamp)
/home/kashif/gluon-ts-PR/src/gluonts/dataset/multivariate_grouper.py:105: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  timestamp + (len(data[FieldName.TARGET]) - 1) * timestamp.freq,
/home/kashif/gluon-ts-PR/src/gluonts/dataset/multivariate_grouper.py:107: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  self.frequency = tim

In [8]:
estimator = DNRIEstimator(
    freq=dataset.metadata.freq,
    context_length=dataset.metadata.prediction_length,
    prediction_length=dataset.metadata.prediction_length,
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    distr_output=StudentTOutput(int(dataset.metadata.feat_static_cat[0].cardinality)),
    mlp_hidden_size=16,
    decoder_hidden=16,
    rnn_hidden_size=16,
    
    batch_size=64,
    num_batches_per_epoch=100,
    trainer_kwargs=dict(max_epochs=1,  accelerator='gpu', gpus=1),
)   

In [9]:
predictor = estimator.train(
    training_data=dataset_train,
    num_workers=4,
    shuffle_buffer_length=1024
)

/home/kashif/.env/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
/home/kashif/gluon-ts-PR/src/gluonts/transform/feature.py:365: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  self._freq_base = start.freq.base
/home/kashif/gluon-ts-PR/src/gluonts/transform/feature.py:365: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  self._freq_base = start.freq.base
GPU available: True, used: True
/home/kashif/gluon-ts-PR/src/gluonts/transform/feature.py:365: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  self._freq_base = start.freq.base
TPU available: False, using: 0 TPU cores
/home/kashif/gluon-ts-PR/src/gluonts/transform/split.py:36: FutureWar

Training: 0it [00:00, ?it/s]

Epoch 0, global step 100: 'train_loss' reached -13.27983 (best -13.27983), saving model to '/home/kashif/pytorch-ts/pts/model/dnri/lightning_logs/version_36/checkpoints/epoch=0-step=100.ckpt' as top 1


In [10]:
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                                 predictor=predictor,
                                                 num_samples=100)

In [11]:
forecasts = list(forecast_it)
tss = list(ts_it)

/home/kashif/gluon-ts-PR/src/gluonts/transform/feature.py:365: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  self._freq_base = start.freq.base
/home/kashif/gluon-ts-PR/src/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  return _shift_timestamp_helper(ts, ts.freq, offset)
/home/kashif/gluon-ts-PR/src/gluonts/transform/feature.py:406: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  ..., i0 : i0 + length * start.freq.n : start.freq.n
/home/kashif/gluon-ts-PR/src/gluonts/transform/feature.py:360: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  assert self._freq_base is None or self._freq_base == start.freq.base, (


In [17]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

In [19]:
agg_metric, _ = evaluator(tss, forecasts, num_series=len(dataset_test))

Running evaluation: 0it [00:00, ?it/s]/home/kashif/gluon-ts-PR/src/gluonts/evaluation/_base.py:313: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  date_before_forecast = forecast.index[0] - forecast.index[0].freq
Running evaluation: 5it [00:00, 85.54it/s]
Running evaluation: 0it [00:00, ?it/s]/home/kashif/gluon-ts-PR/src/gluonts/evaluation/_base.py:313: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  date_before_forecast = forecast.index[0] - forecast.index[0].freq
Running evaluation: 5it [00:00, 104.80it/s]
Running evaluation: 0it [00:00, ?it/s]/home/kashif/gluon-ts-PR/src/gluonts/evaluation/_base.py:313: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  date_before_forecast = forecast.index[0] - forecast.index[0].freq
Running evaluation: 5it [00:00, 108.40it/s]
Running evaluation: 0it [00:00, ?it/s]/home/kashif/gluon-ts-PR/src/gluonts/evaluation/_base.py:313: FutureWarn

In [20]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.01437470598626395
ND: 0.01951843994598358
NRMSE: 0.029499483748858964
MSE: 0.0005756323611780564


In [21]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.007591333333789749
ND-Sum: 0.0102522900613221
NRMSE-Sum: 0.012385791741813566
MSE-Sum: 0.006494472573200861
